## Sans les champs des tables

In [106]:
import mysql.connector
import pygraphviz as pgv

# Création de la connexion
cnx = mysql.connector.connect(
    user='XXX', 
    password='XXX', 
    host='XXX', 
    database='XXX',
    ssl_disabled=False,   #pas forcément besoin
    ssl_verify_cert=False #pas forcément besoin
)    

cursor = cnx.cursor()

# Interroger la BDD pour les relations de clés étrangères
query = """
SELECT TABLE_NAME, COLUMN_NAME, REFERENCED_TABLE_NAME, REFERENCED_COLUMN_NAME 
FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE 
WHERE TABLE_SCHEMA = %s AND REFERENCED_TABLE_NAME IS NOT NULL;
"""
cursor.execute(query, ('nom de notre base de données',))
resultats = cursor.fetchall()

# Initialisation couleur et cluster
couleurs_par_prefixe = {
    'nomPréfixe': 'lightblue',
    'nomPréfixe': 'lightgreen',
}
couleur_target = 'purple'
couleur_defaut = 'blue'


clusters = {
    'nomTargetPourCluster': [],
    'nomTargetPourCluster': [],
    'nomTargetPourCluster': [],
}

# Création d'un graphe -> PyGraphviz
G = pgv.AGraph(strict=True, directed=True)
G.graph_attr.update(rankdir='TB', nodesep='0.5', ranksep='1', size="110,12!")

for (table, column, ref_table, ref_column) in resultats:
    # Exclure les tables target150 de la vue du schéma
    if table.startswith("nomDebutTable") or ref_table.startswith("nomDebutTable"):
        continue

    # Mettre une couleur en fonction des débuts de noms de tables
    couleur_noeud = couleur_defaut
    if table.lower().startswith("nomDebutTable"):
        couleur_noeud = couleurs_par_prefixe['nomDebutTable']
    elif table.lower().startswith("nomDebutTable"):
        couleur_noeud = couleurs_par_prefixe['nomDebutTable']
    elif table.lower() == "nomDebutTable":
        couleur_noeud = couleur_target

    # Vérifier l'appartenance à un cluster
    cluster_appartenance = None
    for cluster_name in clusters.keys():
        if table.startswith(cluster_name):
            cluster_appartenance = cluster_name
            break

    # Faire eb sorte que les clusters fonctionne
    if cluster_appartenance:
        clusters[cluster_appartenance].append(table)
    else:
        G.add_node(table, shape='box', style='filled', fillcolor=couleur_noeud)

    # la couleur par défaut s'il n'est pas déjà ajouté
    if not G.has_node(ref_table):
        G.add_node(ref_table, shape='box', style='filled', fillcolor=couleur_defaut)
    G.add_edge(table, ref_table, label=f"{column} → {ref_column}")

# Ajouter les tables aux clusters spécifiés
for cluster_name, tables in clusters.items():
    with G.subgraph(name='cluster_' + cluster_name) as c:
        c.graph_attr['label'] = cluster_name
        for table in tables:
            c.add_node(table, shape='box', style='filled', fillcolor=couleur_defaut)

# Configuration finale et sauvegarde du schéma
G.layout(prog='dot')
G.draw('schema_sans_champs_noms_bdd.png')

# Fermeture de la connexion
cursor.close()  
cnx.close()
print("Diagramme sans nom champ bien crée")

Diagramme sans nom champ bien crée


## Aves les champs des tables

In [107]:
import mysql.connector
import pygraphviz as pgv

# Création de la connexion
cnx = mysql.connector.connect(
    user='XXX', 
    password='XXX', 
    host='XXX', 
    database='XXX',
    ssl_disabled=False,   #pas forcément besoin
    ssl_verify_cert=False #pas forcément besoin
)    


cursor = cnx.cursor()

# Interroger la BDD pour les relations de clés étrangères
query = """
SELECT TABLE_NAME, COLUMN_NAME, REFERENCED_TABLE_NAME, REFERENCED_COLUMN_NAME 
FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE 
WHERE TABLE_SCHEMA = %s AND REFERENCED_TABLE_NAME IS NOT NULL;
"""
cursor.execute(query, ('nom de notre base de données',))
resultats = cursor.fetchall()

# Récupération des noms des colonnes pour chaque table impliquée
table_columns = {}
tables_involved = set([t[0] for t in resultats] + [t[2] for t in resultats])
for table in tables_involved:
    cursor.execute("""
    SELECT COLUMN_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_SCHEMA = %s AND TABLE_NAME = %s
    """, ('jahia_preprod', table))
    columns = cursor.fetchall()
    table_columns[table] = [col[0] for col in columns]

# Initialisation des couleurs et clusters
couleurs_par_prefixe = {
    'nomPréfixe': 'lightblue',
    'nomPréfixe': 'lightgreen',
}
couleur_target = 'purple'
couleur_defaut = 'blue'

clusters = {
    'nomTargetPourCluster': [],
    'nomTargetPourCluster': [],
    'nomTargetPourCluster': [],
}

# Création du graphe
G = pgv.AGraph(strict=True, directed=True)
#G.graph_attr.update(rankdir='TB', nodesep='0.25', ranksep='1', size="110,12!")

# Ajout des tables et relations au graphe
for (table, column, ref_table, ref_column) in resultats:
    if table.startswith("nomDebutTable") or ref_table.startswith("nomDebutTable"):
        continue
    
    # Construction du label HTML-like avec le nom de la table en rouge
    label = f'<<TABLE BORDER="0" CELLBORDER="0"><TR><TD><FONT COLOR="red">{table}</FONT></TD></TR>'
    for col in table_columns[table]:
        label += f'<TR><TD>{col}</TD></TR>'
    label += '</TABLE>>'
    
    couleur_noeud = couleur_defaut
    if table.lower().startswith("nomDebutTable"):
        couleur_noeud = couleurs_par_prefixe['nomDebutTable']
    elif table.lower().startswith("nomDebutTable"):
        couleur_noeud = couleurs_par_prefixe['nomDebutTable']
    elif table.lower() == "nomDebutTable":
        couleur_noeud = couleur_target

    # Ajout du nœud avec des bords arrondis et remplissage de couleur
    G.add_node(table, label=label, shape='rect', style='filled,rounded', fillcolor=couleur_noeud)

    if not G.has_node(ref_table):
        # Ajout de ref_table avec des bords arrondis et remplissage de couleur, si pas encore ajouté
        G.add_node(ref_table, shape='rect', style='filled,rounded', fillcolor=couleur_defaut)
    G.add_edge(table, ref_table, label=f"{column} → {ref_column}")

# Ajouter les tables aux clusters spécifiés
for cluster_name, tables in clusters.items():
    with G.subgraph(name='cluster_' + cluster_name) as c:
        c.graph_attr['label'] = cluster_name
        for table in tables:
            # Pas besoin de spécifier le label ici, car il a déjà été défini
            c.add_node(table)

# Configuration finale et sauvegarde du schéma
G.layout(prog='dot')
G.draw('schema_bdd.png')

# Fermeture de la connexioncursor.close()  
cnx.close()
print("Diagramme avec noms champs bien crée")

Diagramme avec noms champs bien crée
